In [20]:
from PIL import Image
import numpy as np

def data_to_vgg_input(data):
    if data.mode != "RGB":
        data = data.convert(mode="RGB")
    
    return np.rollaxis(np.asarray(data),2)

def fname_to_vgg_input(fname):
    """ Creates the input for a VGG network from the filename 
        Args: 
            fname (string) : the filename to be parsed
        Returns: 
            (numpy ndarray) : the array to be passed into the VGG network as a single example
    """
    im = Image.open(fname)
    return data_to_vgg_input(im)


In [2]:
import keras
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD

def VGG_16(weights_path=None):
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(3,224,224)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1000, activation='softmax'))

    if weights_path:
        model.load_weights(weights_path)
        
    for i in xrange(5):
        model.pop()

    model2 = Sequential()
    model2.add(ZeroPadding2D((1,1),input_shape=(3,75,75)))
    model2.add(Convolution2D(64, 3, 3, activation='relu'))
    model2.add(ZeroPadding2D((1,1)))
    model2.add(Convolution2D(64, 3, 3, activation='relu'))
    model2.add(MaxPooling2D((2,2), strides=(2,2)))

    model2.add(ZeroPadding2D((1,1)))
    model2.add(Convolution2D(128, 3, 3, activation='relu'))
    model2.add(ZeroPadding2D((1,1)))
    model2.add(Convolution2D(128, 3, 3, activation='relu'))
    model2.add(MaxPooling2D((2,2), strides=(2,2)))

    model2.add(ZeroPadding2D((1,1)))
    model2.add(Convolution2D(256, 3, 3, activation='relu'))
    model2.add(ZeroPadding2D((1,1)))
    model2.add(Convolution2D(256, 3, 3, activation='relu'))
    model2.add(ZeroPadding2D((1,1)))
    model2.add(Convolution2D(256, 3, 3, activation='relu'))
    model2.add(MaxPooling2D((2,2), strides=(2,2)))

    model2.add(ZeroPadding2D((1,1)))
    model2.add(Convolution2D(512, 3, 3, activation='relu'))
    model2.add(ZeroPadding2D((1,1)))
    model2.add(Convolution2D(512, 3, 3, activation='relu'))
    model2.add(ZeroPadding2D((1,1)))
    model2.add(Convolution2D(512, 3, 3, activation='relu'))
    model2.add(MaxPooling2D((2,2), strides=(2,2)))

    model2.add(ZeroPadding2D((1,1)))
    model2.add(Convolution2D(512, 3, 3, activation='relu'))
    model2.add(ZeroPadding2D((1,1)))
    model2.add(Convolution2D(512, 3, 3, activation='relu'))
    model2.add(ZeroPadding2D((1,1)))
    model2.add(Convolution2D(512, 3, 3, activation='relu'))
    model2.add(MaxPooling2D((2,2), strides=(2,2)))

    model2.add(Flatten())
        
    weights = model.get_weights()
    #print weights
    model2.set_weights(weights)
        
    return model2
    
# AUTOLAB_IGNORE_START
truncated_vgg = VGG_16("vgg16_weights.h5")
print truncated_vgg

# AUTOLAB_IGNORE_STOP

Using Theano backend.


In [106]:
import os
import copy

females = map(lambda path: "female_cropped_normalized/"+path, os.listdir("female_cropped_normalized"))
males = map(lambda path: "male_cropped_normalized/"+path, os.listdir("male_cropped_normalized"))
files = copy.copy(females)
files.extend(males)
files = filter(lambda path: path.find("jpg") > -1, files)


In [107]:
def fnames_to_features(fnames, vgg):
    vgg.compile(optimizer="adam", loss='categorical_crossentropy')
    vgg_input = map(lambda fname: fname_to_vgg_input(fname), fnames)
    return vgg.predict(np.asarray(vgg_input))

inp = files

thing2 = fnames_to_features(inp, truncated_vgg)

np.savetxt("features_yearbook.txt", thing2)
labels = map(lambda path: path.find("female")> -1, inp)
np.savetxt("labels_yearbook.txt", np.asarray(labels))
    

In [3]:
features = np.loadtxt("features_yearbook.txt")
labels = np.loadtxt("labels_yearbook.txt", dtype=float).astype(int)

In [3]:
import time

def make_model(X, y):
    from sklearn.linear_model import SGDClassifier
    clf = SGDClassifier(loss="log", penalty="l1")
    clf.fit(X, y)
    return clf

def predict_from_features(X, y, X_te):
    from sklearn.linear_model import SGDClassifier
    clf = SGDClassifier(loss="log", penalty="l1")
    clf.fit(X, y)
    return clf.predict(X_te)

In [4]:
perm = np.random.permutation(len(labels))
perm = perm
numberToTrainOn = int(len(perm)*0.8)
X_tr = features[perm[:numberToTrainOn]]
y_tr = labels[perm[:numberToTrainOn]]
X_va = features[perm[numberToTrainOn:]]
y_va = labels[perm[numberToTrainOn:]]
runs = []

numIters = 100

for i in xrange(numIters):
    start = time.time()
    classifier = make_model(X_tr, y_tr)
    y_p = classifier.predict(X_va)
    end = time.time()
    
    #print "Validation accuracy: {} in {} seconds".format(np.mean(y_p==y_va), end-start)
    runs.append((np.mean(y_p==y_va), end-start))

acc = reduce(lambda x,y:x+y, map(lambda z: z[0], runs))/len(runs)
runtime = reduce(lambda x,y:x+y, map(lambda z: z[1], runs))/len(runs)
print "After {} runs, average acc was {} with average time being {}".format(numIters, acc,runtime)


After 100 runs, average acc was 0.880951949229 with average time being 0.26087900877


In [4]:
features_yrbk = np.loadtxt("features_yearbook.txt")
labels_yrbk = np.loadtxt("labels_yearbook.txt", dtype=float).astype(int)
features_owndata = np.loadtxt("features.txt")
labels_owndata = np.loadtxt("labels.txt", dtype=float).astype(int)

In [26]:
clf_yrbk = make_model(features_yrbk, labels_yrbk)
clf_owndata = make_model(features_owndata, labels_owndata)

vgg = truncated_vgg
vgg.compile(optimizer="adam", loss='categorical_crossentropy')



In [ ]:
#### from autocrop import *
from resizeAndGrayscale import *
import cv2

from PIL import ImageDraw

import matplotlib
matplotlib.use("svg")
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use("ggplot")
import numpy as np

def plot_image(im):
    try:
        plt.imshow(im, aspect='auto')
    except:
        plt.imshow(np.asarray(im[:,:]), aspect='auto')
    
def drawRectangle(drawer, coords, width):
    

def faceCrop(imPath,boxScale=1):
    faceCascade = cv.Load('haarcascade_frontalface_alt.xml')
    
    classifier = clf_owndata
    
    im = Image.open(imPath)
    
    finalImage = Image.open(imPath)
    
    faces = []
    cv_im=pil2cvGrey(im)
    faces=DetectFace(cv_im,faceCascade)
    #print faces
    #plot_image(faces)
    if faces:
        n=1
        drawer = ImageDraw.Draw(finalImage)

        for face in faces:
            faceData = imgCrop(im, face[0],boxScale=boxScale)
            #plot_image(faceData)
            normalized = crop_and_scale_image(faceData, isPath=False)
            #print normalized
            #plot_image(normalized)
            vgg_input = np.array([data_to_vgg_input(normalized)])
            prediction = classifier.predict(vgg.predict(vgg_input))
            prediction = "female" if prediction>0 else "male"
            print prediction
            plot_image(faceData)
            ((x,y,w,h),n) = face
            pt1 = (int(x), int(y))
            pt2 = (int(x + w), int(y + h)) 
            #cv.Rectangle(finalImage, pt1, pt2, cv.RGB(255, 0, 0), 5, 8, 0)
            drawer.rectangle((pt1,pt2), outline="red")
        plot_image(finalImage)
            
            
    else:       
        print 'No faces found:', img

testImage = "testImages/testImage1.jpg"

faceCrop(testImage)
